# Open-Loop Evaluation

In this notebook you are going to evaluate a CNN-based policy to control the SDV with a protocol named *open-loop* evaluation.

**Note: this notebook assumes you've already run the [training notebook](./train.ipynb) and stored your model successfully.**

## What is open-loop evaluation?
In open-loop evaluation we evaluate our model prediction as we follow the annotated ground truth.

In each frame, we compare the predictions of our model against the annotated groud truth. This can be done with different metrics, and we will see few of them in the following.

**Regardless of the metric used, thie evaluation protocol doesn't modify the future locations according to our predictions**

![open-loop](../../images/planning/open-loop.svg)


## What can we use open-loop evaluation for?
Open-loop evaluation can be used for a frame by frame comparison between the expert and the policy. This is extremely useful for debugging the model behaviours and investigate outlier predictions in specific situations (e.g. at crossings or unprotected tunr).

## Is open-loop evaluation enough?
Regardless of the quality of the open-loop results, **this evaluation is not enough** to ensure your model will be able to actually drive on the road (that's where we all want to go in the end). If your model is not in full control of the SDV, you can't really say it will work once the annotated trajectory won't be available anymore.

Before drawing conclusions on our mode we must test it when it is in full control of the SVD, in a setting called **closed-loop**. You can try just that in our [dedicated closed-loop evaluation notebook](./closed_loop_test.ipynb)

In [ ]:
from tempfile import gettempdir
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate
from tqdm import tqdm

from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.geometry import transform_points, angular_distance
from l5kit.visualization import TARGET_POINTS_COLOR, PREDICTED_POINTS_COLOR, draw_trajectory
from l5kit.drivenet.model import DriveNetModel
from l5kit.kinematic import AckermanPerturbation
from l5kit.random import GaussianRandomGenerator

import os

## Prepare data path and load cfg

By setting the `L5KIT_DATA_FOLDER` variable, we can point the script to the folder where the data lies.

Then, we load our config file with relative paths and other configurations (rasteriser, training params...).

In [ ]:
# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = "/tmp/l5kit_data"
dm = LocalDataManager(None)
# get config
cfg = load_config_data("./config.yaml")

## Load the model

In [ ]:
model_path = "/tmp/planning_model.pt"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = torch.load(model_path).to(device)
model = model.eval()

## Load the evaluation data
This is almost the exact same code you've already seen in the [training notebook](./drivenet_train.ipynb). Apart from the different dataset we load, the biggest difference is that **we don't perturb our data here**.

When performing evaluation we're interested in knowing the performance on the annotated data, not on perturbed on.

In [ ]:
# ===== INIT DATASET
eval_cfg = cfg["val_data_loader"]
rasterizer = build_rasterizer(cfg, dm)
eval_zarr = ChunkedDataset(dm.require(eval_cfg["key"])).open()
eval_dataset = EgoDataset(cfg, eval_zarr, rasterizer)
eval_dataloader = DataLoader(eval_dataset, shuffle=eval_cfg["shuffle"], batch_size=eval_cfg["batch_size"], 
                             num_workers=eval_cfg["num_workers"])
print(eval_dataset)

# Evaluation loop

Here, we loop through the data and store predicted and annotated trajectories (positions + yaws).

Note: we're not taking into account availability here. We acknowledge this can reflects in lower metric.

In [ ]:
# ==== EVAL LOOP
position_preds = []
yaw_preds = []

position_gts = []
yaw_gts = []

torch.set_grad_enabled(False)
idx = 0

for data in tqdm(eval_dataloader):
    del data["host_id"]
    result = model(data)
    position_preds.append(result["positions"].detach().cpu().numpy())
    yaw_preds.append(result["yaws"].detach().cpu().numpy())

    position_gts.append(data["target_positions"].detach().cpu().numpy())
    yaw_gts.append(data["target_yaws"].detach().cpu().numpy())
    idx += 1
    if idx == 10:  # TODO remove
        break
    
position_preds = np.concatenate(position_preds)
yaw_preds = np.concatenate(yaw_preds)

position_gts = np.concatenate(position_gts)
yaw_gts = np.concatenate(yaw_gts)

## Quantitative evaluation: ADE, FDE and angle distance

### Positional displacement
[Average Displacement Error (ADE) and Final Displacement Error (FDE)](https://en.wikipedia.org/wiki/Mean_squared_displacement) are standard metrics used to evaluate future predictions for AVs.

We can compute them by comparing predicted and annotated positions, which we have stored in the previous cell.
Additionally, we can plot histograms of their distributions across samples to better capture the variance of our error

### Angle displacement

For the yaw, we can use the Minimum Angle Distance to check the error. Again, we can plot a histogram to inspect the error distribution.

In [ ]:
pos_errors = np.linalg.norm(position_preds - position_gts, axis=-1)

# DISPLACEMENT AT T
plt.plot(np.arange(pos_errors.shape[1]), pos_errors.mean(0), label="Displacement error at T")
plt.legend()
plt.show()

# ADE HIST
plt.hist(pos_errors.mean(-1), bins=100, label="ADE Histogram")
plt.legend()
plt.show()

# FDE HIST
plt.hist(pos_errors[:,-1], bins=100, label="FDE Histogram")
plt.legend()
plt.show()

angle_errors = angular_distance(yaw_preds, yaw_gts).squeeze()

# ANGLE ERROR AT T
plt.plot(np.arange(angle_errors.shape[1]), angle_errors.mean(0), label="Angle error at T")
plt.legend()
plt.show()

# ANGLE ERROR HIST
plt.hist(angle_errors.mean(-1), bins=100, label="Angle Error Histogram")
plt.legend()
plt.show()

## Qualitative evaluation: visual plots

## Visualise results
We can also visualise some images with predicted and annotated trajectories using L5Kit visualisation features.

In this example, we draw 20 images from our dataset and we visualise predicted and annotated trajectories on top of them.

In [ ]:
for frame_number in range(0, len(eval_dataset), len(eval_dataset) // 20):
    
    data = eval_dataloader.dataset[frame_number]
    del data["host_id"]

    data_batch = default_collate([data])
    
    result = model(data_batch)
    predicted_positions = result["positions"].detach().cpu().numpy().squeeze()

    im_ego = rasterizer.to_rgb(data["image"].transpose(1, 2, 0))
    target_positions = data["target_positions"]
    
    predicted_positions = transform_points(predicted_positions, data["raster_from_agent"])
    target_positions = transform_points(target_positions, data["raster_from_agent"])
    
    draw_trajectory(im_ego, predicted_positions, PREDICTED_POINTS_COLOR)
    
    draw_trajectory(im_ego, target_positions, TARGET_POINTS_COLOR)

    plt.imshow(im_ego[::-1])
    plt.axis("off")
    plt.show()

# Qualitative evaluation: visualise the open-loop

To visualise the open loop we can just repeat the same operations for the consecutive frames.

In this example, we show the first 200 frames for our dataset, plotting predicted and annotated trajectories.

**We want to stress this out again: this is an open loop evaluation, we are NOT controlling the AV with our predictions**

In [ ]:
from IPython.display import display, clear_output
import PIL
 
for frame_number in range(200):
    
    data = eval_dataloader.dataset[frame_number]
    del data["host_id"]

    data_batch = default_collate([data])
    
    result = model(data_batch)
    predicted_positions = result["positions"].detach().cpu().numpy().squeeze()

    
    predicted_positions = transform_points(predicted_positions, data["raster_from_agent"])
    target_positions = transform_points(data["target_positions"], data["raster_from_agent"])
    
    im_ego = rasterizer.to_rgb(data["image"].transpose(1, 2, 0))
    draw_trajectory(im_ego, target_positions, TARGET_POINTS_COLOR)
    draw_trajectory(im_ego, predicted_positions, PREDICTED_POINTS_COLOR)
    
    clear_output(wait=True)
    display(PIL.Image.fromarray(im_ego[::-1]))